In [1]:
import chess
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.models import resnet50
from torchvision.models.resnet import Bottleneck
from typing import List, Tuple
import torch.optim as optim
import time
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1

seed = 123456
np.random.seed(seed=seed)

# Разработка модели


In [2]:
class DualHeadChessDataset(Dataset):
    def __init__(self, csv_file: str):
        self.df = pd.read_csv(csv_file)

        self._validate_data()
        self.piece_to_idx = {
            "P": 0,
            "N": 1,
            "B": 2,
            "R": 3,
            "Q": 4,
            "K": 5,
            "p": 6,
            "n": 7,
            "b": 8, 
            "r": 9,
            "q": 10,
            "k": 11,
        }

    def _validate_data(self):
        valid_indices = []
        for idx, row in self.df.iterrows():
            try:
                board = chess.Board(row["fen"])
                move = chess.Move.from_uci(row["move"])
                if move in board.legal_moves:
                    valid_indices.append(idx)
            except:
                continue

        self.df = self.df.loc[valid_indices].reset_index(drop=True)
        print(f"Загружено {len(self.df)} валидных позиций")

    def _board_to_tensor(self, fen: str) -> torch.Tensor:
        board = chess.Board(fen)
        tensor = torch.zeros(20, 8, 8, dtype=torch.float32)

        # Фигуры (плоскости 0-11)
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                row, col = square // 8, square % 8
                piece_idx = self.piece_to_idx[piece.symbol()]
                tensor[piece_idx, row, col] = 1.0

        # Чей ход (плоскость 12)
        tensor[12] = 1.0 if board.turn else 0.0

        # Рокировки (плоскости 13-16)
        castling_rights = [
            board.has_kingside_castling_rights(chess.WHITE),
            board.has_queenside_castling_rights(chess.WHITE),
            board.has_kingside_castling_rights(chess.BLACK),
            board.has_queenside_castling_rights(chess.BLACK),
        ]
        for i, has_right in enumerate(castling_rights):
            if has_right:
                tensor[13 + i] = 1.0

        # Взятие на проходе (плоскость 17)
        if board.ep_square is not None:
            row, col = board.ep_square // 8, board.ep_square % 8
            tensor[17, row, col] = 1.0

        # Счетчик полуходов (плоскость 18)
        tensor[18] = board.halfmove_clock / 50.0

        # Номер хода (плоскость 19)
        tensor[19] = board.fullmove_number / 500.0

        return tensor

    def _move_to_dual_labels(self, move_uci: str, fen: str) -> tuple:
        board = chess.Board(fen)
        move = chess.Move.from_uci(move_uci)

        from_square = move.from_square

        # Для to_square учитываем превращения
        # ДИАПАЗОН ДОЛЖЕН БЫТЬ 0-67 (68 классов)
        # print(move.promotion)
        if move.promotion:
            # Кодируем превращения: 64-68
            to_square = 64 + (move.promotion - 1)  # 64, 65, 66, 67, 68
        else:
            to_square = move.to_square  # 0-63

        # Проверяем, что to_square в правильном диапазоне
        if to_square >= 69:
            raise ValueError(f"Некорректный to_square: {to_square} для хода {move_uci}")

        return from_square, to_square

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        fen = row["fen"]
        move = row["move"]

        board_tensor = self._board_to_tensor(fen)
        from_label, to_label = self._move_to_dual_labels(move, fen)

        return board_tensor, from_label, to_label

In [3]:
class ChessDataSplitter:
    def __init__(
        self,
        csv_file,
        train_ratio=0.7,
        val_ratio=0.15,
        test_ratio=0.15,
        random_state=42,
    ):
        print(train_ratio, val_ratio, test_ratio)
        assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, (
            "Сумма долей должна быть равна 1"
        )

        self.csv_file = csv_file
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio
        self.random_state = random_state

        # Загружаем полный dataset
        self.full_dataset = DualHeadChessDataset(csv_file)

    def split_data(self):
        """Разбивает данные на train/val/test"""
        # Получаем индексы
        n_total = len(self.full_dataset)
        indices = list(range(n_total))

        # Первое разбиение: отделяем test
        train_val_idx, test_idx = train_test_split(
            indices,
            test_size=self.test_ratio,
            random_state=self.random_state,
            shuffle=True,
        )

        # Второе разбиение: разделяем train и val
        train_idx, val_idx = train_test_split(
            train_val_idx,
            test_size=self.val_ratio / (self.train_ratio + self.val_ratio),
            random_state=self.random_state,
            shuffle=True,
        )

        # Создаем Subset datasets
        train_dataset = Subset(self.full_dataset, train_idx)
        val_dataset = Subset(self.full_dataset, val_idx)
        test_dataset = Subset(self.full_dataset, test_idx)

        print(f"Разбиение завершено:")
        print(
            f"Train: {len(train_dataset)} samples ({len(train_dataset) / n_total * 100:.1f}%)"
        )
        print(
            f"Val: {len(val_dataset)} samples ({len(val_dataset) / n_total * 100:.1f}%)"
        )
        print(
            f"Test: {len(test_dataset)} samples ({len(test_dataset) / n_total * 100:.1f}%)"
        )

        return train_dataset, val_dataset, test_dataset

In [4]:
class DualHeadChessModel(nn.Module):
    def __init__(self, device="cpu"):
        super().__init__()
        self.device = device

        # ResNet-50 backbone с модификациями
        self.backbone = resnet50(pretrained=True)
        
        # 1. Заменяем первый слой для 20 каналов
        self.backbone.conv1 = nn.Conv2d(20, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # 2. Заменяем BatchNorm слои (важно!)
        # Получаем все дочерние модули
        backbone_layers = list(self.backbone.children())
        
        # 3. Убираем последние слои (avgpool и classifier)
        self.backbone = nn.Sequential(*backbone_layers[:-2])
        
        # 4. Добавляем свой adaptive pooling
        self.adaptive_pool = nn.AdaptiveAvgPool2d((8, 8))
        
        # 5. Добавляем BatchNorm и Dropout для стабилизации
        self.from_bn1 = nn.BatchNorm1d(256)
        self.from_dropout = nn.Dropout(0.3)
        
        self.to_bn1 = nn.BatchNorm2d(256)
        self.to_bn2 = nn.BatchNorm1d(128)
        self.to_dropout = nn.Dropout(0.3)

        # Голова для исходной клетки
        self.from_fc1 = nn.Linear(2048 * 8 * 8, 256)
        self.from_output = nn.Linear(256, 64)

        # Голова для целевой клетки
        self.to_conv = nn.Conv2d(2048 + 64, 256, 3, padding=1)
        self.to_fc1 = nn.Linear(256 * 8 * 8, 128)
        self.to_output = nn.Linear(128, 69)

        # Инициализация
        self._initialize_weights()
        
        # Перенос на устройство
        self.to(device)

    def _initialize_weights(self):
        """Более безопасная инициализация"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, board_tensor, from_square=None):
        # Проверка входных данных
        if torch.isnan(board_tensor).any():
            print("Обнаружены NaN во входных данных!")
            
        # Общие признаки через ResNet
        x = self.backbone(board_tensor)
        shared_features = self.adaptive_pool(x)
        
        # Проверка промежуточных значений
        if torch.isnan(shared_features).any():
            print("NaN после backbone!")

        # Голова для исходной клетки
        from_flat = shared_features.reshape(shared_features.size(0), -1)
        from_hidden = F.relu(self.from_fc1(from_flat))
        from_hidden = self.from_bn1(from_hidden)
        from_hidden = self.from_dropout(from_hidden)
        from_logits = self.from_output(from_hidden)
        
        # Используем log_softmax для стабильности с NLLLoss
        from_probs = F.log_softmax(from_logits, dim=1)

        if from_square is None:
            return from_probs, None

        # Голова для целевой клетки
        from_onehot = torch.zeros(shared_features.size(0), 64, 8, 8, device=self.device)
        from_onehot[torch.arange(shared_features.size(0)), from_square, :, :] = 1

        to_input = torch.cat([shared_features, from_onehot], dim=1)
        to_features = F.relu(self.to_conv(to_input))
        to_features = self.to_bn1(to_features)
        
        to_flat = to_features.reshape(to_features.size(0), -1)
        to_hidden = F.relu(self.to_fc1(to_flat))
        to_hidden = self.to_bn2(to_hidden)
        to_hidden = self.to_dropout(to_hidden)
        to_logits = self.to_output(to_hidden)
        to_probs = F.log_softmax(to_logits, dim=1)

        return from_probs, to_probs

In [5]:
class ChessTrainer:
    def __init__(self
                 ,model
                 ,train_loader
                 ,val_loader
                 ,test_loader
                 ,device="cpu"
                 ,lr=0.01
                 ,weight_decay = 0.1
                ):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.lr = lr
        self.weight_decay = weight_decay

        # Функции потерь
        self.from_criterion = nn.NLLLoss()
        self.to_criterion = nn.NLLLoss()

        # Оптимизатор
        self.optimizer = optim.AdamW(model.parameters(), lr=self.lr, weight_decay = self.weight_decay, amsgrad=True)

        # Планировщик learning rate
        self.scheduler = optim.lr_scheduler.StepLR(
            self.optimizer, step_size=5, gamma=0.5
        )

        # История обучения
        self.history = {
            "train_loss": [],
            "val_loss": [],
            "train_from_acc": [],
            "val_from_acc": [],
            "train_to_acc": [],
            "val_to_acc": [],
            "learning_rate": [],
        }

    def train_epoch(self):
        """Одна эпоха обучения"""
        self.model.train()
        total_loss = 0
        total_from_correct = 0
        total_to_correct = 0
        total_samples = 0

        train_bar = tqdm(self.train_loader, desc="Training")

        for batch_idx, (boards, from_targets, to_targets) in enumerate(train_bar):
            # Перемещаем данные на device
            boards = boards.to(self.device)
            from_targets = from_targets.to(self.device)
            to_targets = to_targets.to(self.device)

            # Обнуляем градиенты
            self.optimizer.zero_grad()

            # Прямой проход
            print(boards)
            from_probs, to_probs = self.model(boards, from_targets)

            # Вычисляем потери
            print(from_probs.shape, from_targets.shape, to_probs.shape, to_targets.shape)
            print(from_probs, from_targets, to_probs, to_targets)
            from_loss = self.from_criterion(from_probs, from_targets)
            to_loss = self.to_criterion(to_probs, to_targets)
            print(from_loss.shape, to_loss.shape)
            loss = from_loss + to_loss

            # Обратный проход
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

            # Шаг оптимизатора
            self.optimizer.step()

            # Статистика
            total_loss += loss.item()
            batch_size = boards.size(0)
            total_samples += batch_size

            # Точность для from-square
            from_preds = torch.argmax(from_probs, dim=1)+1
            from_correct = (from_preds == from_targets).sum().item()
            total_from_correct += from_correct

            # Точность для to-square
            to_preds = torch.argmax(to_probs, dim=1)+1
            to_correct = (to_preds == to_targets).sum().item()
            total_to_correct += to_correct

            # Обновляем progress bar
            train_bar.set_postfix(
                {
                    "Loss": f"{loss.item():.4f}",
                    "From Acc": f"{from_correct / batch_size:.3f}",
                    "To Acc": f"{to_correct / batch_size:.3f}",
                }
            )

        # Вычисляем средние метрики за эпоху
        avg_loss = total_loss / len(self.train_loader)
        from_acc = total_from_correct / total_samples
        to_acc = total_to_correct / total_samples

        return avg_loss, from_acc, to_acc

    def validate(self):
        """Валидация модели"""
        self.model.eval()
        total_loss = 0
        total_from_correct = 0
        total_to_correct = 0
        total_samples = 0

        with torch.no_grad():
            for boards, from_targets, to_targets in tqdm(
                self.val_loader, desc="Validation"
            ):
                boards = boards.to(self.device)
                from_targets = from_targets.to(self.device)
                to_targets = to_targets.to(self.device)

                # Прямой проход
                from_probs, to_probs = self.model(boards, from_targets)

                # Потери
                from_loss = self.from_criterion(from_probs, from_targets)
                to_loss = self.to_criterion(to_probs, to_targets)
                loss = from_loss + to_loss

                total_loss += loss.item()
                batch_size = boards.size(0)
                total_samples += batch_size

                # Точность
                from_preds = torch.argmax(from_probs, dim=1)+1
                # print(from_probs, from_preds, from_targets)
                from_correct = (from_preds == from_targets).sum().item()
                total_from_correct += from_correct

                to_preds = torch.argmax(to_probs, dim=1)+1
                to_correct = (to_preds == to_targets).sum().item()
                total_to_correct += to_correct

        avg_loss = total_loss / len(self.val_loader)
        from_acc = total_from_correct / total_samples
        to_acc = total_to_correct / total_samples

        return avg_loss, from_acc, to_acc

    def train(self, num_epochs=50, early_stopping_patience=10):
        """Полный цикл обучения"""
        best_val_loss = float("inf")
        patience_counter = 0

        print("Начало обучения...")
        print(f"Используется устройство: {self.device}")
        print(f"Размер тренировочного набора: {len(self.train_loader.dataset)}")
        print(f"Размер валидационного набора: {len(self.val_loader.dataset)}")

        for epoch in range(num_epochs):
            print(f"\nЭпоха {epoch + 1}/{num_epochs}")
            print("-" * 50)

            # Обучение
            train_loss, train_from_acc, train_to_acc = self.train_epoch()

            # Валидация
            val_loss, val_from_acc, val_to_acc = self.validate()

            # Обновление learning rate
            self.scheduler.step()
            current_lr = self.optimizer.param_groups[0]["lr"]

            # Сохраняем историю
            self.history["train_loss"].append(train_loss)
            self.history["val_loss"].append(val_loss)
            self.history["train_from_acc"].append(train_from_acc)
            self.history["val_from_acc"].append(val_from_acc)
            self.history["train_to_acc"].append(train_to_acc)
            self.history["val_to_acc"].append(val_to_acc)
            self.history["learning_rate"].append(current_lr)

            # Выводим результаты
            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            print(
                f"Train From Acc: {train_from_acc:.4f} | Val From Acc: {val_from_acc:.4f}"
            )
            print(f"Train To Acc: {train_to_acc:.4f} | Val To Acc: {val_to_acc:.4f}")
            print(f"Learning Rate: {current_lr:.6f}")

            # Сохранение лучшей модели
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                self.save_model(f"best_chess_model_{self.model.__class__.__name__}.pth")
                print("Сохранена лучшая модель!")
            else:
                patience_counter += 1
                print(
                    f"Early stopping: {patience_counter}/{early_stopping_patience}"
                )

            # Early stopping
            if patience_counter >= early_stopping_patience:
                print("Ранняя остановка!")
                break

        print("\nОбучение завершено!")

    def evaluate(self):
        """Финальная оценка на тестовом наборе"""
        print("\nОценка на тестовом наборе...")
        self.model.eval()

        total_from_correct = 0
        total_to_correct = 0
        total_full_correct = 0
        total_samples = 0

        with torch.no_grad():
            for boards, from_targets, to_targets in tqdm(
                self.test_loader, desc="Testing"
            ):
                boards = boards.to(self.device)
                from_targets = from_targets.to(self.device)
                to_targets = to_targets.to(self.device)

                # Предсказание from-square
                from_probs, _ = self.model(boards)
                from_preds = torch.argmax(from_probs, dim=1)

                # Предсказание to-square (используя правильные from-square для честной оценки)
                _, to_probs = self.model(boards, from_targets)
                to_preds = torch.argmax(to_probs, dim=1)

                # Подсчет правильных предсказаний
                from_correct = (from_preds == from_targets).sum().item()
                to_correct = (to_preds == to_targets).sum().item()
                full_correct = (
                    ((from_preds == from_targets) & (to_preds == to_targets))
                    .sum()
                    .item()
                )

                total_from_correct += from_correct
                total_to_correct += to_correct
                total_full_correct += full_correct
                total_samples += boards.size(0)

        from_acc = total_from_correct / total_samples
        to_acc = total_to_correct / total_samples
        full_acc = total_full_correct / total_samples

        print(f"\nРезультаты на тестовом наборе:")
        print(f"From-square Accuracy: {from_acc:.4f}")
        print(f"To-square Accuracy: {to_acc:.4f}")
        print(f"Full Move Accuracy: {full_acc:.4f}")

        return from_acc, to_acc, full_acc

    def save_model(self, path):
        """Сохранение модели"""
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "history": self.history,
            },
            path,
        )

    def load_model(self, path):
        """Загрузка модели"""
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        self.history = checkpoint["history"]

    def plot_training_history(self):
        """Визуализация истории обучения"""
        import matplotlib.pyplot as plt

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

        # Потери
        ax1.plot(self.history["train_loss"], label="Train Loss")
        ax1.plot(self.history["val_loss"], label="Val Loss")
        ax1.set_title("Потери")
        ax1.set_xlabel("Эпоха")
        ax1.set_ylabel("Loss")
        ax1.legend()
        ax1.grid(True)

        # From-square точность
        ax2.plot(self.history["train_from_acc"], label="Train From Acc")
        ax2.plot(self.history["val_from_acc"], label="Val From Acc")
        ax2.set_title("From-square Точность")
        ax2.set_xlabel("Эпоха")
        ax2.set_ylabel("Accuracy")
        ax2.legend()
        ax2.grid(True)

        # To-square точность
        ax3.plot(self.history["train_to_acc"], label="Train To Acc")
        ax3.plot(self.history["val_to_acc"], label="Val To Acc")
        ax3.set_title("To-square Точность")
        ax3.set_xlabel("Эпоха")
        ax3.set_ylabel("Accuracy")
        ax3.legend()
        ax3.grid(True)

        # Learning rate
        ax4.plot(self.history["learning_rate"])
        ax4.set_title("Learning Rate")
        ax4.set_xlabel("Эпоха")
        ax4.set_ylabel("LR")
        ax4.grid(True)

        plt.tight_layout()
        plt.savefig("training_history.png", dpi=300, bbox_inches="tight")
        plt.show()

In [36]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Используется устройство: {device}")

    torch.cuda.empty_cache()

    # Загрузка данных
    print("Загрузка данных...")
    splitter = ChessDataSplitter(
        test_ratio=0.01,
        val_ratio=0.2 * (1 - 0.01),
        train_ratio=0.8 * (1 - 0.01),
        csv_file="fens_training_set.csv",
    )
    train_dataset, val_dataset, test_dataset = splitter.split_data()
    _, test_to_print = train_test_split(pd.read_csv("fens_training_set.csv"), test_size=0.01, random_state=42)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=0)

    # Модель
    print("Инициализация модели...")
    model = DualHeadChessModel(device=device)
    
    # Загрузка весов модели
    model_path = "best_chess_model_DualHeadChessModel.pth"
    print(f"Загрузка весов модели из {model_path}...")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    # Предсказание на тестовом наборе
    print("Предсказание на тестовом наборе...")
    predictions, true_moves, comparison_results = predict_and_compare(model, test_loader, device, test_to_print)
    
    # Вывод результатов сравнения
    print("\n" + "="*60)
    print("РЕЗУЛЬТАТЫ СРАВНЕНИЯ НА ТЕСТОВОМ НАБОРЕ")
    print("="*60)
    
    # Статистика точности
    from_correct = sum(1 for result in comparison_results if result['from_correct'])
    to_correct = sum(1 for result in comparison_results if result['to_correct'])
    full_correct = sum(1 for result in comparison_results if result['full_correct'])
    total_samples = len(comparison_results)
    
    from_accuracy = from_correct / total_samples * 100
    to_accuracy = to_correct / total_samples * 100
    full_accuracy = full_correct / total_samples * 100
    
    print(f"\nТочность предсказаний:")
    print(f"From-square: {from_accuracy:.2f}% ({from_correct}/{total_samples})")
    print(f"To-square:   {to_accuracy:.2f}% ({to_correct}/{total_samples})")
    print(f"Полный ход:  {full_accuracy:.2f}% ({full_correct}/{total_samples})")
    
    # Вывод примеров предсказаний
    print(f"\nПримеры предсказаний (первые 10):")
    print("-" * 100)
    print(f"{'№':<3} {'Предсказание':<12} {'Истинный':<12} {'From':<6} {'To':<6} {'Ход':<6} {'Позиция'}")
    print("-" * 100)
    
    for i, result in enumerate(comparison_results[:10]):
        status_from = "+" if result['from_correct'] else "-"
        status_to = "+" if result['to_correct'] else "-"
        status_full = "+" if result['full_correct'] else "-"
        
        print(f"{i+1:<3} {result['predicted_move']:<12} {result['true_move']:<12} "
              f"{status_from:<6} {status_to:<6} {status_full:<6} {result['position']}")
        
        
    print(f"\nПодробные примеры ошибок:")
    print("-" * 60)
    error_examples = [r for r in comparison_results if not r['full_correct']][:5]
    for i, error in enumerate(error_examples):
        print(f"Ошибка {i+1}:")
        print(f"  Предсказано: {error['predicted_move']} "
              f"(from: {error['predicted_from']}, to: {error['predicted_to']})")
        print(f"  Истинный:    {error['true_move']} "
              f"(from: {error['true_from']}, to: {error['true_to']})")
        print()


def predict_and_compare(model, test_loader, device, test_dataset):
    """
    Предсказание ходов на тестовом наборе и сравнение с истинными значениями
    """
    model.eval()
    all_predictions = []
    all_true_moves = []
    comparison_results = []
    
    with torch.no_grad():
        for batch_idx, (boards, from_targets, to_targets) in enumerate(tqdm(test_loader, desc="Testing")):
            boards = boards.to(device)
            from_targets = from_targets.to(device)
            to_targets = to_targets.to(device)
            
            # Предсказание from-square
            from_probs, _ = model(boards)
            from_preds = torch.argmax(from_probs, dim=1)
            
            # Предсказание to-square (используя предсказанные from-square)
            _, to_probs = model(boards, from_preds)
            to_preds = torch.argmax(to_probs, dim=1)
            
            # Конвертация индексов в шахматные обозначения
            batch_size = boards.size(0)
            for i in range(batch_size):
                # Предсказанные значения
                pred_from_idx = from_preds[i].cpu().item()
                pred_to_idx = to_preds[i].cpu().item()
                pred_from_square = index_to_square(pred_from_idx)
                pred_to_square = index_to_square(pred_to_idx)
                predicted_move = f"{pred_from_square}{pred_to_square}"
                
                # Истинные значения
                true_from_idx = from_targets[i].cpu().item()
                true_to_idx = to_targets[i].cpu().item()
                true_from_square = index_to_square(true_from_idx)
                true_to_square = index_to_square(true_to_idx)
                true_move = f"{true_from_square}{true_to_square}"
                
                # Сравнение
                from_correct = (pred_from_idx == true_from_idx)
                to_correct = (pred_to_idx == true_to_idx)
                full_correct = from_correct and to_correct
                
                sample_idx = batch_idx * test_loader.batch_size + i
                fen = test_dataset.loc[test_dataset.index[sample_idx], 'fen']

                # Сохраняем результаты
                all_predictions.append(predicted_move)
                all_true_moves.append(true_move)
                
                comparison_results.append({
                    'sample_id': batch_idx * test_loader.batch_size + i,
                    'predicted_move': predicted_move,
                    'true_move': true_move,
                    'predicted_from': pred_from_square,
                    'predicted_to': pred_to_square,
                    'true_from': true_from_square,
                    'true_to': true_to_square,
                    'predicted_from_idx': pred_from_idx,
                    'predicted_to_idx': pred_to_idx,
                    'true_from_idx': true_from_idx,
                    'true_to_idx': true_to_idx,
                    'from_correct': from_correct,
                    'to_correct': to_correct,
                    'full_correct': full_correct,
                    'position': fen
                })
    
    return all_predictions, all_true_moves, comparison_results


def index_to_square(index):
    """
    Конвертирует индекс (0-63) в шахматное обозначение клетки (a1-h8)
    """
    
    if index < 64:
        files = "abcdefgh"
        ranks = "12345678"
        return f"{files[index % 8]}{ranks[index // 8]}"
    elif index < 68:
        promotions = ["q", "r", "b", "n"]
        return f"promote_{promotions[index - 64]}"
    else:
        return "special"

In [37]:
if __name__ == "__main__":
    main()

Используется устройство: cuda
Загрузка данных...
0.792 0.198 0.01
Загружено 268549 валидных позиций
Разбиение завершено:
Train: 212690 samples (79.2%)
Val: 53173 samples (19.8%)
Test: 2686 samples (1.0%)
Инициализация модели...


f:\dlml\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\dlml\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Загрузка весов модели из best_chess_model_DualHeadChessModel.pth...
Предсказание на тестовом наборе...


Testing: 100%|██████████| 269/269 [00:17<00:00, 15.19it/s]



РЕЗУЛЬТАТЫ СРАВНЕНИЯ НА ТЕСТОВОМ НАБОРЕ

Точность предсказаний:
From-square: 39.35% (1057/2686)
To-square:   18.02% (484/2686)
Полный ход:  14.85% (399/2686)

Примеры предсказаний (первые 10):
----------------------------------------------------------------------------------------------------
№   Предсказание Истинный     From   To     Ход    Позиция
----------------------------------------------------------------------------------------------------
1   a1d1         d3d4         -      -      -      1r3rk1/1ppq1ppp/p3p3/3pP3/6n1/1QNPP3/PP3PPP/R4RK1 w - - 0 1
2   f3e5         f3e5         +      +      +      r1bq1rk1/bpp2pp1/3p1n1p/p2Nn3/4P3/2P2N2/PPB2PPP/R1BQ1RK1 w - - 0 1
3   f1e2         f1d3         +      -      -      2rq2k1/pp5p/3p2p1/3Npr2/1PP2b2/7P/P4P2/R2QKB1R w K - 0 1
4   g2f3         g2f3         +      +      +      r3kb1r/ppp4p/2nq1nP1/1B1pp3/Q2P4/2P2b2/PP3PPP/RNB1K2R w KQkq - 0 1
5   g1f3         g1f3         +      +      +      rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/